# Lab 8
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72

****
****
****

## Module importing

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from IPython.display import display

In [4]:
import time 

In [5]:
sns.set_style('darkgrid')
%matplotlib inline

## Algorithm implementation

In [6]:
class Rand:
    """
    Генерація випадкового числа за заданим законом розподілу
    """
    @staticmethod
    def exp(time_mean):
        a = 0
        while a == 0:
            a = np.random.rand()
        return -time_mean * np.log(a)
    
    @staticmethod
    def unif(time_min, time_max):
        a = 0
        while a == 0:
            a = np.random.rand()
        a = time_min + a * (time_max - time_min)
        return a
    
    @staticmethod
    def norm(time_mean, time_deviation):
        return np.random.normal(loc=time_mean, scale=time_deviation)

In [7]:
class Position:
    def __init__(self, name, num_of_markers=0, description=None):
        self.num_of_markers = num_of_markers
        self.name = name
        self.description = description
        self.type = 'Position'
        
    def add_markers(self, amount_of_added_markers):
        self.num_of_markers += amount_of_added_markers
        
    def remove_markers(self, amount_of_removed_markers):
        self.num_of_markers -= amount_of_removed_markers
        
    def __repr__(self):
        return f'Name: {self.name}' + ("\nDescription: " + self.description if self.description else "-") + '\n'

In [8]:
class Transition:
    def __init__(self, name, priority=0, probability=None, distribution=None, delay_params=None, description=None):
        self.name = name
        self.priority = priority
        self.probability = probability
        self.distribution = distribution
        self.delay_params = delay_params
        self.description = description
        self.type = 'Transition'
        
        self.input_arcs = []
        self.output_arcs = []
        
    def is_available(self):
        return all(arc.is_available() for arc in self.input_arcs)
    
    def make_a_transition(self):
        for arc in self.input_arcs:
            arc.move_from()
        
        if self.distribution is not None: self.distribution(*self.delay_params)
        
        for arc in self.output_arcs:
            arc.move_to()
    
    def __repr__(self):
        return f'Name: {self.name}' + ("\nDescription: " + self.description if self.description else "-") + '\n'

In [9]:
class Arc:
    def __init__(self, start, end, multiplicity=1):
        self.start = start
        self.end = end
        self.multiplicity = multiplicity
        self.type = 'Arc'
        
        if isinstance(end, Transition):
            end.input_arcs.append(self)  
        elif isinstance(start, Transition):
            start.output_arcs.append(self)
        
    def is_available(self):
        return self.start.num_of_markers >= self.multiplicity
    
    def move_from(self):
        self.start.remove_markers(self.multiplicity)
        
    def move_to(self):
        self.end.add_markers(self.multiplicity)
        
    def __repr__(self):
        return f'{self.start.name}({self.start.type}) ---> {self.end.name}({self.end.type})\n'

In [10]:
class Model:
    def __init__(self, transitions, positions, arcs):
        self.transitions = transitions
        self.positions = positions
        self.arcs = arcs
        self.tcurr = 0
        
        diff_stats = {'avg_markers': 0, 'max_markers': 0, 'min_markers': np.inf, 'result_markers': 0, 'n_cnt': 0}
        self.positions_stats = {position.name: diff_stats.copy() for position in self.positions}
        
    def simulate(self, time_modeling):
        self.time_modeling = time_modeling
        
        while self.tcurr < self.time_modeling:
            we_have_available_transition = any(transition.is_available() for transition in self.transitions)
            
            if not we_have_available_transition:
                break
                
            resolved_transitions = self.resolve_conflict()
            
            self.do_calc()
            for transition in resolved_transitions:
                if transition.is_available():
                    transition.make_a_transition()
            self.tcurr += 1
            
        return self.print_results()
            
    def resolve_conflict(self):
        resolved_list = []
        conflict_list = []
        
        # розділяємо нормальні стани та конфліктні
        for position in self.positions:
            output_arcs_for_position = [*filter(lambda arc: arc.start is position, self.arcs)]
            resolved = [*filter(lambda trans: trans.is_available(),
                                map(lambda arc: arc.end, output_arcs_for_position))
                       ]
            if len(resolved) > 1:
                conflict_list.append(resolved)
                    
        conf_list_flatten = set(np.array(conflict_list).flatten().tolist())
        for position in self.positions:
            output_arcs_for_position = [*filter(lambda arc: arc.start is position, self.arcs)]
            resolved = [*filter(lambda trans: trans.is_available(),
                                map(lambda arc: arc.end, output_arcs_for_position))
                       ]
            
            if not conf_list_flatten.intersection(resolved):
                resolved_list += resolved
        del conf_list_flatten

        # вирішуємо конфлікти
        for conflict in conflict_list:
            # конфлікт за ймовірністю
            if conflict[0].probability is not None:
                p = [c.probability for c in conflict]
                resolved = np.random.choice(conflict, p=p)
            # конфлікт за пріоритетом
            elif len(np.unique([c.priority for c in conflict if c.priority is not None])) > 1:
                priority = [c.priority for c in conflict]
                resolved = conflict[np.argmax(priority)]
            else:
                resolved = np.random.choice(conflict)
            
            resolved_list += [resolved]
            
#         np.random.shuffle(resolved_list)
        return list(set(resolved_list))
    
    def do_calc(self):
        for position in self.positions:
#             print(position, position.num_of_markers, '\n\n')
            self.positions_stats[position.name]['avg_markers'] += position.num_of_markers
            self.positions_stats[position.name]['max_markers'] = max(self.positions_stats[position.name]['max_markers'], position.num_of_markers)
            self.positions_stats[position.name]['min_markers'] = min(self.positions_stats[position.name]['min_markers'], position.num_of_markers)
            self.positions_stats[position.name]['result_markers'] = position.num_of_markers
            self.positions_stats[position.name]['n_cnt'] += 1
#         print('-----------------')
        
    def prepare_stats(self):
        for position in self.positions:
            self.positions_stats[position.name]['avg_markers'] /= self.positions_stats[position.name]['n_cnt']
            del self.positions_stats[position.name]['n_cnt']
    
    def print_results(self):
        self.prepare_stats()
        
        rows = []
        for position in self.positions:
            self.positions_stats[position.name]['Name'] = position.name
            self.positions_stats[position.name]['Description'] = position.description if position.description else "-"
            
            rows.append(self.positions_stats[position.name])
            
        df = pd.DataFrame(rows, columns=list(self.positions_stats[position.name].keys()))
        display(df)
        return df

## Task 1
Згідно протоколу передачу одного повідомлення від вузла А до вузла В описують такі події: запит від А на передачу в В, позитивна відповідь від вузла В вузлу А, передача повідомлення з А в В, відправка повідомлення вузлом А, отримання повідомлення вузлом В, відправка сигналу про успішне отримання повідомлення вузлом В, відправка сигналу про успішне отримання повідомлення вузлом В.<br>
Двостороння передача означає, що обмін повідомленнями відбувається одночасно в обох напрямках. Через те, що у вузлі може зберігатись тільки одне  повідомлення, може виникати тупикова ситуація, коли два вузли здійснили відправку і «зависли» в очікуванні підтвердження отримання повідомлення іншим вузлом. Для запобігання тупикової ситуації, використовується керуючий сигнал, що надає дозвіл на відправку повідомлення тільки одному з двох вузлів.

<img src="img/task1.jpg">

In [13]:
class SimpleModel1:
    def __init__(self):
        # Задамо позиції
        p1 = Position('P1', num_of_markers=1, description='Генерація повідомлення від А для В')
        p2 = Position('P2', num_of_markers=0, description=None)
        p3 = Position('P3', num_of_markers=0, description=None)
        p4 = Position('P4', num_of_markers=0, description='Кількість успішно отриманих повідомлень вузлом В від вузла А')
        p5 = Position('P5', num_of_markers=1, description='Вільність каналу')
        p6 = Position('P6', num_of_markers=1, description='Генерація повідомлення від В для А')
        p7 = Position('P7', num_of_markers=0, description=None)
        p8 = Position('P8', num_of_markers=0, description=None)
        p9 = Position('P9', num_of_markers=0, description='Кількість успішно отриманих повідомлень вузлом А від вузла В')
        
        # Задамо переходи
        t1 = Transition('T1', description='Запит від А на передачу В')
        t2 = Transition('T2', description='Відправлення повідомлення вузлом А')
        t3 = Transition('T3', description='Отримання повідомлення вузлом В')
        t4 = Transition('T4', description='Запит від В на передачу А')
        t5 = Transition('T5', description='Відправлення повідомлення вузлом В')
        t6 = Transition('T6', description='Отримання повідомлення вузлом А')
        
        # Задамо дуги
        a_p1_t1 = Arc(start=p1, end=t1)
        a_p5_t1 = Arc(start=p5, end=t1)
        a_t1_p1 = Arc(start=t1, end=p1)
        a_t1_p2 = Arc(start=t1, end=p2)
        
        a_p2_t2 = Arc(start=p2, end=t2)
        a_t2_p3 = Arc(start=t2, end=p3)
        
        a_p3_t3 = Arc(start=p3, end=t3)
        a_t3_p5 = Arc(start=t3, end=p5)
        a_t3_p4 = Arc(start=t3, end=p4)
        
        a_p6_t4 = Arc(start=p6, end=t4)
        a_p5_t4 = Arc(start=p5, end=t4)
        a_t4_p6 = Arc(start=t4, end=p6)
        a_t4_p7 = Arc(start=t4, end=p7)
        
        a_p7_t5 = Arc(start=p7, end=t5)
        a_t5_p8 = Arc(start=t5, end=p8)
        
        a_p8_t6 = Arc(start=p8, end=t6)
        a_t6_p5 = Arc(start=t6, end=p5)
        a_t6_p9 = Arc(start=t6, end=p9)
        
        # Створимо мережу Петрі
        petri_network = Model(positions=[p1,p2,p3,p4,p5,p6,p7,p8,p9],
                              transitions=[t1,t2,t3,t4,t5,t6],
                              arcs=[a_p1_t1,a_p5_t1,a_t1_p1,a_t1_p2,a_p2_t2,
                                    a_t2_p3,a_p3_t3,a_t3_p5,a_t3_p4,a_p6_t4,
                                    a_p5_t4,a_t4_p6,a_t4_p7,a_p7_t5,a_t5_p8,
                                    a_p8_t6,a_t6_p5,a_t6_p9])
        self.result = petri_network.simulate(1000)

In [14]:
_ = SimpleModel1()

,avg_markers,max_markers,min_markers,result_markers,Name,Description
0,1.000,1,1,1,P1,Генерація повідомлення від А для В
1,0.164,1,0,0,P2,-
2,0.164,1,0,0,P3,-
3,79.514,164,0,164,P4,Кількість успішно отриманих повідомлень вузлом...
4,0.334,1,0,1,P5,Вільність каналу
5,1.000,1,1,1,P6,Генерація повідомлення від В для А
6,0.169,1,0,0,P7,-
7,0.169,1,0,0,P8,-
8,86.653,169,0,169,P9,Кількість успішно отриманих повідомлень вузлом...


## Task 2
Процес Producer постачає задачі для виконання Consumer і розміщує їх в буфер (операція put). Процес Consumer виймає задачі з буфера (операція take) і обробляє їх. Оскільки буфер має обмеження n, то при досягненні максимального значення припиняється робота процесу Producer. Якщо задач в буфері немає, то робота процесу Consumer припиняється.<br>
За результатами моделювання потрібно оцінити середнє значення буфера.

<img src="img/task2.jpg">

In [16]:
class SimpleModel2:
    def __init__(self):
        # Задамо гіпер-параметри
        n_buffer = 32
        
        # Задамо позиції
        p1 = Position('P1', num_of_markers=1, description='Producer постачає задачу')
        p2 = Position('P2', num_of_markers=n_buffer, description='Buffer size')
        p3 = Position('P3', num_of_markers=0, description='Buffer')
        p4 = Position('P4', num_of_markers=0, description=None)
        p5 = Position('P5', num_of_markers=0, description='Кількість оброблених задач')
        
        # Задамо переходи
        t1 = Transition('T1', description='Розміщення задачі (PUT)')
        t2 = Transition('T2', description='Consumer дістає задачу (TAKE)')
        t3 = Transition('T3', description='Consumer оброблює задачу')
        
        # Задамо дуги
        a_p1_t1 = Arc(start=p1, end=t1)
        a_t1_p1 = Arc(start=t1, end=p1)
        a_p2_t1 = Arc(start=p2, end=t1)
        a_t1_p3 = Arc(start=t1, end=p3)
        
        a_p3_t2 = Arc(start=p3, end=t2)
        a_t2_p2 = Arc(start=t2, end=p2)
        a_t2_p4 = Arc(start=t2, end=p4)
        
        a_p4_t3 = Arc(start=p4, end=t3)
        
        a_t3_p5 = Arc(start=t3, end=p5)
        
        # Створимо мережу Петрі
        petri_network = Model(positions=[p1,p2,p3,p4,p5],
                              transitions=[t1,t2,t3],
                              arcs=[a_p1_t1,a_t1_p1,a_p2_t1,a_t1_p3,
                                    a_p3_t2,a_t2_p2,a_t2_p4,a_p4_t3,
                                    a_t3_p5])
        self.result = petri_network.simulate(1000)

In [17]:
_ = SimpleModel2()

,avg_markers,max_markers,min_markers,result_markers,Name,Description
0,1.000,1,1,1,P1,Producer постачає задачу
1,31.001,32,31,31,P2,Buffer size
2,0.999,1,0,1,P3,Buffer
3,0.998,1,0,1,P4,-
4,497.503,997,0,997,P5,Кількість оброблених задач


## Task 3
Задачі різних типів надходять у багатопроцесорну систему. Задачі першого типу вимагають усі процесори системи, задачі другого типу – третину всіх обчислювальних ресурсів, а задачі третього типу – половину ресурсів.<br>
За результатами моделювання потрібно оцінити співвідношення кількості виконаних завдань.

<img src="img/task3.jpg">

In [24]:
class SimpleModel3:
    def __init__(self):
        # Задамо гіпер-параметри
        n_res = 50
        
        # Задамо позиції
        p1 = Position('P1', num_of_markers=1, description='Надходження задач 1-го типу')
        p2 = Position('P2', num_of_markers=1, description='Надходження задач 2-го типу')
        p3 = Position('P3', num_of_markers=1, description='Надходження задач 3-го типу')
        p9 = Position('P9', num_of_markers=1, description='Надходження задач 4-го типу')
        p4 = Position('P4', num_of_markers=n_res, description='Загальний ресурс')
        p5 = Position('P5', num_of_markers=0, description='Кількість оброблених задач 1-го типу')
        p6 = Position('P6', num_of_markers=0, description='Кількість оброблених задач 2-го типу')
        p7 = Position('P7', num_of_markers=0, description='Кількість оброблених задач 3-го типу')
        p8 = Position('P8', num_of_markers=0, description='Кількість оброблених задач 4-го типу')
        
        # Задамо переходи
        t1 = Transition('T1', description='Обробка задач 1-го типу')
        t2 = Transition('T2', description='Обробка задач 2-го типу')
        t3 = Transition('T3', description='Обробка задач 3-го типу')
        t4 = Transition('T4', description='Обробка задач 4-го типу')
        
        # Задамо дуги
        a_p1_t1 = Arc(start=p1, end=t1)
        a_t1_p1 = Arc(start=t1, end=p1)
        a_t1_p5 = Arc(start=t1, end=p5)
        a_t1_p4 = Arc(start=t1, end=p4, multiplicity=n_res)
        a_p4_t1 = Arc(start=p4, end=t1, multiplicity=n_res)
        
        a_p2_t2 = Arc(start=p2, end=t2)
        a_t2_p2 = Arc(start=t2, end=p2)
        a_t2_p6 = Arc(start=t2, end=p6)
        a_t2_p4 = Arc(start=t2, end=p4, multiplicity=n_res//3)
        a_p4_t2 = Arc(start=p4, end=t2, multiplicity=n_res//3)
        
        a_p3_t3 = Arc(start=p3, end=t3)
        a_t3_p3 = Arc(start=t3, end=p3)
        a_t3_p7 = Arc(start=t3, end=p7)
        a_t3_p4 = Arc(start=t3, end=p4, multiplicity=n_res//2)
        a_p4_t3 = Arc(start=p4, end=t3, multiplicity=n_res//2)
        
        a_p9_t4 = Arc(start=p9, end=t4)
        a_t4_p9 = Arc(start=t4, end=p9)
        a_t4_p8 = Arc(start=t4, end=p8)
        a_t4_p4 = Arc(start=t4, end=p4, multiplicity=n_res//4)
        a_p4_t4 = Arc(start=p4, end=t4, multiplicity=n_res//4)
        
        
        # Створимо мережу Петрі
        petri_network = Model(positions=[p1,p2,p3,p4,p5,p6,p7,p8,p9],
                              transitions=[t1,t2,t3,t4],
                              arcs=[a_p1_t1,a_t1_p1,a_t1_p5,a_t1_p4,a_p4_t1,
                                    a_p2_t2,a_t2_p2,a_t2_p6,a_t2_p4,a_p4_t2,
                                    a_p3_t3,a_t3_p3,a_t3_p7,a_t3_p4,a_p4_t3,
                                    a_p9_t4,a_t4_p9,a_t4_p8,a_t4_p4,a_p4_t4])
        self.result = petri_network.simulate(1000)

In [25]:
sm3 = SimpleModel3()

,avg_markers,max_markers,min_markers,result_markers,Name,Description
0,1.000,1,1,1,P1,Надходження задач 1-го типу
1,1.000,1,1,1,P2,Надходження задач 2-го типу
2,1.000,1,1,1,P3,Надходження задач 3-го типу
3,50.000,50,50,50,P4,Загальний ресурс
4,129.503,261,0,261,P5,Кількість оброблених задач 1-го типу
5,130.692,260,0,260,P6,Кількість оброблених задач 2-го типу
6,118.889,238,0,238,P7,Кількість оброблених задач 3-го типу
7,120.416,240,0,240,P8,Кількість оброблених задач 4-го типу
8,1.000,1,1,1,P9,Надходження задач 4-го типу


In [29]:
t = sm3.result[sm3.result['Description'].isin({'Кількість оброблених задач 1-го типу',
                                               'Кількість оброблених задач 2-го типу',
                                               'Кількість оброблених задач 3-го типу',
                                               'Кількість оброблених задач 4-го типу'}
                                             )].reset_index(drop=True)

In [30]:
t['Ratio_of_result_markers'] = t['result_markers'] / t['result_markers'].sum()

In [31]:
t

,avg_markers,max_markers,min_markers,result_markers,Name,Description,Ratio_of_result_markers
0,129.503,261,0,261,P5,Кількість оброблених задач 1-го типу,0.261261
1,130.692,260,0,260,P6,Кількість оброблених задач 2-го типу,0.260260
2,118.889,238,0,238,P7,Кількість оброблених задач 3-го типу,0.238238
3,120.416,240,0,240,P8,Кількість оброблених задач 4-го типу,0.240240


## Conclusion

В даній лабораторній роботі було реалізовано алгоритм імітації роботи базової мережі Петрі, досліджено властивості її елементів та зібрано певні статистичні дані.